# Interpretable Regression

In this notebook we will fit regression explainable boosting machine (EBM), LinearRegression, and RegressionTree models. After fitting them, we will use their glassbox nature to understand their global and local explanations.

This notebook can be found in our [**_examples folder_**](https://github.com/interpretml/interpret/tree/develop/docs/interpret/python/examples) on GitHub.

In [ ]:
# install interpret if not already installed
try:
    import interpret
except ModuleNotFoundError:
    !pip install --quiet interpret scikit-learn

In [ ]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from interpret import show
from interpret.perf import RegressionPerf

from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

dataset = fetch_california_housing()
X = dataset.data
y = dataset.target
names = dataset.feature_names

seed = 42
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

<h2>Explore the dataset</h2>

In [ ]:
from interpret import show
from interpret.data import Marginal

marginal = Marginal(names).explain_data(X_train, y_train, name='Train Data')
show(marginal)

<h2>Train the Explainable Boosting Machine (EBM)</h2>

In [ ]:
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree

ebm = ExplainableBoostingRegressor(names, interactions=3)
ebm.fit(X_train, y_train)

<h2>EBMs are glassbox models, so we can edit them</h2>

In [ ]:
# post-process monotonize the MedInc feature
ebm.monotonize("MedInc", increasing=True)

<h2>Global Explanations: What the model learned overall</h2>

In [ ]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<h2>Local Explanations: How an individual prediction was made</h2>

In [ ]:
ebm_local = ebm.explain_local(X_test[:5], y_test[:5], name='EBM')
show(ebm_local, 0)

<h2>Evaluate EBM performance</h2>

In [ ]:
ebm_perf = RegressionPerf(ebm, names).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

<h2>Let's test out a few other Explainable Models</h2>

In [ ]:
from interpret.glassbox import LinearRegression, RegressionTree

lr = LinearRegression(names)
lr.fit(X_train, y_train)

rt = RegressionTree(names, random_state=seed)
rt.fit(X_train, y_train)

<h2>Compare performance using the Dashboard</h2>

In [ ]:
lr_perf = RegressionPerf(lr, names).explain_perf(X_test, y_test, name='Linear Regression')
show(lr_perf)

In [ ]:
rt_perf = RegressionPerf(rt, names).explain_perf(X_test, y_test, name='Regression Tree')
show(rt_perf)

<h2>Glassbox: All of our models have global and local explanations</h2>

In [ ]:
lr_global = lr.explain_global(name='Linear Regression')
show(lr_global)

In [ ]:
rt_global = rt.explain_global(name='Regression Tree')
show(rt_global)